In [ ]:
# ============================================
# CELLA 1: Upload File Excel (qualsiasi nome)
# ============================================

from google.colab import files
from pathlib import Path
import shutil

print("📤 CARICA IL TUO FILE EXCEL")
print("="*60)
print("Può avere qualsiasi nome (es: video.xlsx, prompts.xlsx, etc.)")
print("Struttura richiesta:")
print("  • Colonna 'Prompt' → Testo del prompt")
print("  • Colonna 'NomeFile' → Nome video (senza .mp4)")
print("="*60)
print()

uploaded = files.upload()

if not uploaded:
    print("❌ Nessun file caricato!")
else:
    # Prendi il primo file caricato
    filename = list(uploaded.keys())[0]

    # Verifica che sia un Excel
    if not (filename.endswith('.xlsx') or filename.endswith('.xls')):
        print(f"❌ File '{filename}' non è un Excel!")
        print("   Carica un file .xlsx o .xls")
    else:
        # Rinomina in prompts.xlsx per compatibilità con script
        if filename != 'prompts.xlsx':
            shutil.copy(filename, 'prompts.xlsx')
            print(f"✅ File '{filename}' caricato e rinominato in 'prompts.xlsx'")
        else:
            print(f"✅ File '{filename}' caricato!")

        # Mostra preview
        import pandas as pd
        try:
            df = pd.read_excel('prompts.xlsx')
            total = len(df)
            print(f"\n📊 CONTENUTO FILE:")
            print(f"   • Righe totali: {total}")

            if 'Prompt' in df.columns and 'NomeFile' in df.columns:
                print(f"   • Colonne trovate: ✅ Prompt, ✅ NomeFile")
                print(f"\n📋 PREVIEW (primi 3):")
                for i, row in df.head(3).iterrows():
                    prompt_preview = str(row['Prompt'])[:50]
                    if len(str(row['Prompt'])) > 50:
                        prompt_preview += "..."
                    print(f"   {i+1}. {row['NomeFile']}")
                    print(f"      → {prompt_preview}")
                if total > 3:
                    print(f"   ... e altri {total-3} prompt")
            else:
                print(f"   ⚠️ ATTENZIONE: Colonne non corrette!")
                print(f"   Colonne trovate: {', '.join(df.columns)}")
                print(f"   Colonne richieste: 'Prompt', 'NomeFile'")
        except Exception as e:
            print(f"⚠️ Errore lettura file: {e}")

        print("\n" + "="*60)
        print("✅ PRONTO! Ora puoi eseguire:")
        print("   • CELLA 3 → Login (solo prima volta)")
        print("   • CELLA 4 → Genera video")
        print("="*60)



📤 CARICA IL TUO FILE EXCEL
Può avere qualsiasi nome (es: video.xlsx, prompts.xlsx, etc.)
Struttura richiesta:
  • Colonna 'Prompt' → Testo del prompt
  • Colonna 'NomeFile' → Nome video (senza .mp4)



Saving truff 13.xlsx to truff 13.xlsx
✅ File 'truff 13.xlsx' caricato e rinominato in 'prompts.xlsx'

📊 CONTENUTO FILE:
   • Righe totali: 174
   • Colonne trovate: ✅ Prompt, ✅ NomeFile

📋 PREVIEW (primi 3):
   1. SCENE 01_01
      → Photorealistic 50-year-old Italian mechanic salt-p...
   2. SCENE 01_02
      → Photorealistic POV holding paper invoice high pric...
   3. SCENE 01_03
      → Photorealistic stressed car owner sitting in drive...
   ... e altri 171 prompt

✅ PRONTO! Ora puoi eseguire:
   • CELLA 3 → Login (solo prima volta)
   • CELLA 4 → Genera video


In [ ]:
# CELLA 2: Installazione dipendenze
print("⚙️ Installazione dipendenze in corso...")

# Installa browser e driver
!apt-get update -qq > /dev/null 2>&1
!apt-get install -y chromium-browser chromium-chromedriver > /dev/null 2>&1

# Installa librerie Python
!pip install -q playwright pandas openpyxl

# Installa browser Playwright
!playwright install chromium
!playwright install-deps

print("✅ Installazione completata!")

⚙️ Installazione dipendenze in corso...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 20.0 MB/s eta 0:00:00
(node:1380) [DEP0169] DeprecationWarning: `url.parse()` behavior is not standardized and prone to errors that have security implications. Use the WHATWG URL API instead. CVEs are not issued for `url.parse()` vulnerabilities.
(Use `node --trace-deprecation ...` to show where the warning was created)
167.3 MiB [] 0% 340.0s167.3 MiB [] 0% 46.5s167.3 MiB [] 0% 19.0s167.3 MiB [] 0% 15.6s167.3 MiB [] 0% 10.6s167.3 MiB [] 1% 5.7s167.3 MiB [] 2% 4.2s167.3 MiB [] 3% 3.4s167.3 MiB [] 4% 3.4s167.3 MiB [] 5% 3.0s167.3 MiB [] 6% 2.7s167.3 MiB [] 7% 2.5s167.3 MiB [] 8% 2.3s167.3 MiB [] 9% 2.2s167.3 MiB [] 10% 2.1s167.3 MiB [] 11% 2.0s167.3 MiB [] 12% 2.0s167.3 MiB [] 13% 1.9s167.3 MiB [] 14% 1.8s167.3 MiB [] 15% 1.8s167.3 MiB [] 17% 1.8s167.3 MiB [] 18% 1.7s167.3 MiB [] 19% 1.7s167.3 MiB [] 20% 1.6s167.3 MiB [] 21% 1.6s167.3 MiB [] 22% 1.6s167.3 MiB [] 23% 1.5s167.3 MiB [] 24% 1.4s1

In [ ]:
# ============================================
# CELLA 3: LOGIN E SALVA COOKIES (CORRETTA)
# ============================================

import getpass
from playwright.async_api import async_playwright
import asyncio
import json
from pathlib import Path

print("🔐 CONFIGURAZIONE LOGIN DIGEN.AI")
print("="*60)
print("Inserisci le credenziali del tuo account Digen.ai")
print("(Se non hai account, creane uno su https://digen.ai)")
print("="*60)
print()

EMAIL = input("📧 Email: ")
PASSWORD = getpass.getpass("🔒 Password: ")

print("\n✅ Credenziali salvate!")
print("\n🔄 Eseguo login e salvo sessione...")

async def save_login():
    cookies_file = Path("digen_cookies.json")
    debug_folder = Path("debug_screenshots")
    debug_folder.mkdir(exist_ok=True)

    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=True,
            args=['--no-sandbox', '--disable-dev-shm-usage']
        )
        context = await browser.new_context(
            viewport={'width': 1920, 'height': 1080},
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        )
        page = await context.new_page()

        try:
            print("🌐 Apertura pagina login...")
            await page.goto("https://digen.ai/signin", timeout=30000)
            await asyncio.sleep(5)

            # Screenshot iniziale
            await page.screenshot(path=debug_folder / "1_login_page.png", full_page=True)
            print("📸 Screenshot pagina: debug_screenshots/1_login_page.png")

            # ✅ NUOVO: Usa selettori più flessibili per trovare i campi
            print("🔍 Cerco campo email...")

            # Prova tutti questi selettori
            email_selectors = [
                'input[placeholder*="email" i]',
                'input[placeholder*="Email" i]',
                'input[placeholder*="tua email" i]',
                'input[type="email"]',
                'input[name="email"]',
                '#email',
                'input[aria-label*="email" i]',
                'input:not([type="password"])',  # Primo input che non è password
            ]

            email_field = None
            for selector in email_selectors:
                try:
                    count = await page.locator(selector).count()
                    if count > 0:
                        email_field = page.locator(selector).first
                        print(f"✅ Campo email trovato con: {selector}")
                        break
                except:
                    continue

            if not email_field:
                # Fallback: prendi il primo input visibile
                try:
                    all_inputs = await page.locator('input:visible').all()
                    if len(all_inputs) >= 2:
                        email_field = all_inputs[0]
                        print(f"✅ Campo email trovato (primo input visibile)")
                except:
                    pass

            if not email_field:
                print("❌ Campo email non trovato!")
                print("📋 Prova questi metodi alternativi:")
                print("   1. Usa 'Continua con Google' invece")
                print("   2. Esporta cookies manualmente da browser")
                return False

            print(f"📧 Inserisco email: {EMAIL}")
            await email_field.click()
            await asyncio.sleep(0.5)
            await email_field.fill(EMAIL)
            await asyncio.sleep(1)

            # Screenshot dopo email
            await page.screenshot(path=debug_folder / "2_email_filled.png", full_page=True)

            print("🔍 Cerco campo password...")

            # Prova tutti questi selettori per password
            password_selectors = [
                'input[type="password"]',
                'input[placeholder*="password" i]',
                'input[placeholder*="Password" i]',
                'input[name="password"]',
                '#password',
                'input[aria-label*="password" i]',
            ]

            password_field = None
            for selector in password_selectors:
                try:
                    count = await page.locator(selector).count()
                    if count > 0:
                        password_field = page.locator(selector).first
                        print(f"✅ Campo password trovato con: {selector}")
                        break
                except:
                    continue

            if not password_field:
                # Fallback: prendi il secondo input visibile
                try:
                    all_inputs = await page.locator('input:visible').all()
                    if len(all_inputs) >= 2:
                        password_field = all_inputs[1]
                        print(f"✅ Campo password trovato (secondo input visibile)")
                except:
                    pass

            if not password_field:
                print("❌ Campo password non trovato!")
                return False

            print("🔒 Inserisco password...")
            await password_field.click()
            await asyncio.sleep(0.5)
            await password_field.fill(PASSWORD)
            await asyncio.sleep(1)

            # Screenshot form completo
            await page.screenshot(path=debug_folder / "3_form_filled.png", full_page=True)
            print("📸 Screenshot form compilato")

            print("🔍 Cerco bottone submit...")

            # Prova tutti questi selettori per il bottone
            submit_selectors = [
                'button:has-text("Accedi")',
                'button:has-text("accedi")',
                'button:has-text("ACCEDI")',
                'button:has-text("Sign in")',
                'button:has-text("Login")',
                'button:has-text("Log in")',
                'button[type="submit"]',
                'input[type="submit"]',
                'button:visible:not(:has-text("Google"))',  # Bottone che non dice "Google"
            ]

            submit_btn = None
            for selector in submit_selectors:
                try:
                    count = await page.locator(selector).count()
                    if count > 0:
                        submit_btn = page.locator(selector).first
                        print(f"✅ Bottone trovato con: {selector}")
                        break
                except:
                    continue

            if not submit_btn:
                # Fallback: cerca qualsiasi bottone blu visibile
                try:
                    all_buttons = await page.locator('button:visible').all()
                    for btn in all_buttons:
                        text = await btn.inner_text()
                        if text and "google" not in text.lower():
                            submit_btn = btn
                            print(f"✅ Bottone trovato: {text}")
                            break
                except:
                    pass

            if not submit_btn:
                print("❌ Bottone submit non trovato!")
                return False

            print("🖱️ Clicco submit...")
            await submit_btn.click()

            print("⏳ Attendo risposta (8 secondi)...")
            await asyncio.sleep(8)

            # Screenshot dopo submit
            await page.screenshot(path=debug_folder / "4_after_submit.png", full_page=True)
            print("📸 Screenshot dopo submit")

            # Verifica URL
            current_url = page.url
            print(f"📍 URL attuale: {current_url}")

            # Controlla se siamo ancora sulla pagina di login
            if "signin" in current_url or "login" in current_url:
                print("\n⚠️ Ancora sulla pagina login!")

                # Controlla se ci sono messaggi di errore
                try:
                    error_text = await page.locator('text=/error|errore|incorrect|sbagliato/i').first.text_content(timeout=2000)
                    print(f"   Errore visibile: {error_text}")
                except:
                    pass

                print("\n❌ Login fallito!")
                print("   Possibili cause:")
                print("   • Credenziali errate")
                print("   • Captcha/Verifica richiesta")
                print("   • Rate limit")
                print("\n📸 Controlla gli screenshot in debug_screenshots/")
                return False

            # Login riuscito! Salva cookies
            cookies = await context.cookies()

            if len(cookies) < 1:
                print("⚠️ Nessun cookie ricevuto (strano...)")
                # Ma proviamo comunque a salvare

            with open(cookies_file, 'w') as f:
                json.dump(cookies, f, indent=2)

            print(f"\n✅ LOGIN RIUSCITO!")
            print(f"💾 Salvati {len(cookies)} cookies")
            print(f"📝 File: digen_cookies.json")

            # Mostra alcuni cookies (senza valori sensibili)
            print(f"\n🍪 Cookies salvati:")
            for cookie in cookies[:5]:
                print(f"   • {cookie['name']} (domain: {cookie['domain']})")
            if len(cookies) > 5:
                print(f"   ... e altri {len(cookies)-5}")

            return True

        except Exception as e:
            print(f"\n❌ ERRORE: {e}")
            try:
                await page.screenshot(path=debug_folder / "error.png", full_page=True)
                print("📸 Screenshot errore salvato")
            except:
                pass
            return False

        finally:
            await browser.close()

# Esegui login
success = await save_login()

if success:
    print("\n" + "="*60)
    print("🎉 SETUP COMPLETATO!")
    print("="*60)
    print("\n💡 Ora esegui la CELLA 4 per generare i video!")
    print("   I cookies verranno riutilizzati automaticamente")
else:
    print("\n" + "="*60)
    print("❌ SETUP FALLITO")
    print("="*60)
    print("\n📋 METODI ALTERNATIVI:")
    print("\n1️⃣ LOGIN CON GOOGLE (più facile):")
    print("   • Vai su https://digen.ai")
    print("   • Clicca 'Continua con Google'")
    print("   • Esporta cookies con extension browser")
    print("\n2️⃣ ESPORTA COOKIES MANUALMENTE:")
    print("   • Installa 'EditThisCookie' (Chrome extension)")
    print("   • Fai login manualmente su digen.ai")
    print("   • Esporta cookies")
    print("   • Carica su Colab come digen_cookies.json")
    print("\n3️⃣ PROVA DI NUOVO:")
    print("   • Verifica email/password corrette")
    print("   • Riesegui questa cella")


🔐 CONFIGURAZIONE LOGIN DIGEN.AI
Inserisci le credenziali del tuo account Digen.ai
(Se non hai account, creane uno su https://digen.ai)

📧 Email: alnini3@gmail.com
🔒 Password: ··········

✅ Credenziali salvate!

🔄 Eseguo login e salvo sessione...
🌐 Apertura pagina login...
📸 Screenshot pagina: debug_screenshots/1_login_page.png
🔍 Cerco campo email...
✅ Campo email trovato con: input[placeholder*="email" i]
📧 Inserisco email: alnini3@gmail.com
🔍 Cerco campo password...
✅ Campo password trovato con: input[type="password"]
🔒 Inserisco password...
📸 Screenshot form compilato
🔍 Cerco bottone submit...
✅ Bottone trovato con: button:has-text("Sign in")
🖱️ Clicco submit...
⏳ Attendo risposta (8 secondi)...
📸 Screenshot dopo submit
📍 URL attuale: https://digen.ai/en/explore

✅ LOGIN RIUSCITO!
💾 Salvati 10 cookies
📝 File: digen_cookies.json

🍪 Cookies salvati:
   • _ga (domain: .digen.ai)
   • g_state (domain: digen.ai)
   • intercom-id-wgkan4nt (domain: .digen.ai)
   • intercom-device-id-wgkan4n

In [ ]:
# ============================================
# CELLA 4: GENERAZIONE VIDEO - GROK (Space 44998) fix download
# ============================================

from google.colab import drive
try:
    drive.mount('/content/drive')
except:
    print("Drive già montato")

import pandas as pd
from playwright.async_api import async_playwright
import asyncio
import json
from pathlib import Path
from collections import defaultdict

# ⚙️ CONFIGURAZIONE
MODEL = "grok"
BATCH_SIZE = 2
PAUSE_BETWEEN_PROMPTS = 20
MAX_DOWNLOAD_RETRIES = 2
MAX_GENERATION_WAIT = 500
DOWNLOAD_FOLDER_NAME = "video_generati_grok"

DURATION = "6s"
ASPECT_RATIO = "3:2"
RESOLUTION = "1080P"

# 🐞 Debug (metti True per vedere tutti i dettagli)
DEBUG_MODE = False

async def automate_digen_grok_batch():
    BASE_URL = "https://digen.ai/en/space/44998"
    print(f"🎬 Modalità: {MODEL.upper()} - Space 44998")
    print(f"⚙️ Configurazione: {DURATION}, {ASPECT_RATIO}, {RESOLUTION}\n")

    print("📂 Lettura file Excel...")
    df = pd.read_excel('prompts.xlsx')
    total_prompts = len(df)
    print(f"✅ Trovati {total_prompts} prompt\n")

    download_folder = Path(f"/content/drive/MyDrive/{DOWNLOAD_FOLDER_NAME}")
    download_folder.mkdir(exist_ok=True, parents=True)
    print(f"📁 Cartella download: {download_folder}\n")

    cookies_file = Path("digen_cookies.json")
    tracking_file = Path(f"video_tracking_{MODEL}.json")
    failed_file = Path(f"failed_prompts_{MODEL}.json")

    if tracking_file.exists():
        with open(tracking_file, 'r') as f:
            completed_prompts = set(json.load(f))
    else:
        completed_prompts = set()

    if failed_file.exists():
        with open(failed_file, 'r') as f:
            failed_prompts = json.load(f)
    else:
        failed_prompts = {}

    async def close_any_dialogs(page):
        try:
            await page.keyboard.press('Escape')
            await asyncio.sleep(0.5)
            overlay = page.locator('[data-slot="dialog-overlay"][data-state="open"]')
            if await overlay.count() > 0:
                await page.keyboard.press('Escape')
                await asyncio.sleep(1)
            await page.wait_for_selector('[data-slot="dialog-overlay"][data-state="open"]', state='hidden', timeout=5000)
        except:
            pass

    async def select_grok_model(page):
        """🎯 Clicca su bottone modello e seleziona 'Grok Video' dal menu popup"""
        try:
            if DEBUG_MODE:
                print(f"   🎯 Seleziono modello Grok...")
            await close_any_dialogs(page)
            await asyncio.sleep(2)

            all_buttons = await page.locator('button:visible').all()

            model_button = None
            for btn in all_buttons:
                try:
                    text = await btn.text_content()
                    if not text:
                        continue

                    text_clean = text.strip()
                    box = await btn.bounding_box()

                    if not box:
                        continue

                    if 900 < box['y'] < 1100:
                        if 3 <= len(text_clean) <= 10 and any(c.isalpha() for c in text_clean):
                            if text_clean.lower() not in ['auto', 'create']:
                                if any(c.isdigit() for c in text_clean) or text_clean.isalpha():
                                    model_button = btn
                                    if DEBUG_MODE:
                                        print(f"   ✅ Trovato bottone modello: '{text_clean}' @ pos:({int(box['x'])},{int(box['y'])})")
                                    break
                except:
                    continue

            if not model_button:
                return False

            await model_button.click()
            await asyncio.sleep(3)

            candidates = []

            selectors = [
                'button:visible',
                '[role="menuitem"]:visible',
                '[role="option"]:visible',
                'div:visible',
                'span:visible',
                'li:visible',
                'a:visible'
            ]

            for selector in selectors:
                try:
                    elements = await page.locator(selector).all()

                    for elem in elements:
                        try:
                            text = await elem.text_content()
                            if not text:
                                continue

                            text_clean = text.strip()
                            text_lower = text_clean.lower()

                            if 'grok' not in text_lower:
                                continue

                            box = await elem.bounding_box()
                            if not box:
                                continue

                            if 3 <= len(text_clean) <= 50:
                                priority = 0
                                if 'grok video' in text_lower:
                                    priority = 1
                                elif text_lower == 'grok':
                                    priority = 2
                                else:
                                    priority = 3

                                candidates.append((elem, text_clean, priority, len(text_clean), box['width'] * box['height']))
                        except:
                            continue
                except:
                    continue

            if not candidates:
                await page.keyboard.press('Escape')
                await asyncio.sleep(0.5)
                return False

            candidates.sort(key=lambda c: (c[2], c[3], c[4]))

            best_elem, best_text, best_priority, best_len, best_area = candidates[0]

            try:
                await best_elem.click(timeout=5000)
                await asyncio.sleep(2)
                return True
            except Exception as click_err:
                try:
                    clickable_child = best_elem.locator('button, [role="button"], a').first
                    if await clickable_child.count() > 0:
                        await clickable_child.click(timeout=5000)
                        await asyncio.sleep(2)
                        return True
                except:
                    pass

                await page.keyboard.press('Escape')
                await asyncio.sleep(0.5)
                return False

        except Exception as e:
            return False

    async def set_duration(page, duration):
        try:
            await close_any_dialogs(page)
            await asyncio.sleep(1)

            all_buttons = await page.locator('button:visible').all()

            for btn in all_buttons:
                try:
                    text = await btn.text_content()
                    if not text:
                        continue

                    text_clean = text.strip()
                    box = await btn.bounding_box()

                    if not box or not (900 < box['y'] < 1100):
                        continue

                    if 's' in text_clean.lower() and len(text_clean) < 5:
                        if duration in text_clean:
                            return True

                        await btn.click()
                        await asyncio.sleep(3)

                        all_visible = await page.locator('*:visible').all()
                        for elem in all_visible:
                            try:
                                text = await elem.text_content()
                                if text and duration == text.strip():
                                    await elem.click()
                                    await asyncio.sleep(1)
                                    return True
                            except:
                                continue

                        await page.keyboard.press('Escape')
                        await asyncio.sleep(0.5)
                        break
                except:
                    continue

            return False

        except Exception as e:
            return False

    async def set_aspect_ratio(page, ratio):
        try:
            await close_any_dialogs(page)
            await asyncio.sleep(1)

            all_buttons = await page.locator('button:visible').all()

            for btn in all_buttons:
                try:
                    text = await btn.text_content()
                    if not text:
                        continue

                    text_clean = text.strip()
                    box = await btn.bounding_box()

                    if not box or not (900 < box['y'] < 1100):
                        continue

                    if ':' in text_clean or 'auto' in text_clean.lower():
                        if ratio in text_clean:
                            return True

                        await btn.click()
                        await asyncio.sleep(3)

                        all_visible = await page.locator('*:visible').all()
                        for elem in all_visible:
                            try:
                                text = await elem.text_content()
                                if text and ratio == text.strip():
                                    await elem.click()
                                    await asyncio.sleep(1)
                                    return True
                            except:
                                continue

                        await page.keyboard.press('Escape')
                        await asyncio.sleep(0.5)
                        break
                except:
                    continue

            return False

        except Exception as e:
            return False

    async def set_resolution(page, resolution):
        """🎞️ MULTI-STRATEGIA per attivare 1080P"""
        try:
            print(f"   🎞️ Imposto risoluzione {resolution}...")
            await close_any_dialogs(page)
            await asyncio.sleep(1)

            # ➡️ TROVA bottone risoluzione principale
            all_buttons = await page.locator('button:visible').all()
            res_button = None
            current_res = "?"

            for btn in all_buttons:
                try:
                    text = await btn.text_content()
                    if not text:
                        continue
                    text_clean = text.strip()
                    box = await btn.bounding_box()
                    if not box or not (900 < box['y'] < 1100):
                        continue
                    if 'p' in text_clean.lower() and len(text_clean) < 8:
                        res_button = btn
                        current_res = text_clean
                        print(f"   🔍 Bottone: '{current_res}'")
                        if '1080' in text_clean:
                            print(f"   ✅ {resolution} già attivo")
                            return True
                        break
                except:
                    continue

            if not res_button:
                print(f"   ⚠️ Bottone non trovato")
                return True

            # ➡️ APRI menu
            print(f"   📂 Apro menu...")
            await res_button.click(force=True)
            await asyncio.sleep(2)

            async def try_click_1080():
                """🔥 6 STRATEGIE"""

                # 1️⃣ Radio button
                try:
                    radio = page.get_by_role("radio", name="1080P").first
                    if await radio.count() > 0:
                        await radio.click(timeout=3000)
                        print(f"   ✅ [1/6] Radio button")
                        await asyncio.sleep(1)
                        return True
                except:
                    pass

                # 2️⃣ Button
                try:
                    btn = page.get_by_role("button", name="1080P").first
                    if await btn.count() > 0:
                        await btn.click(timeout=3000)
                        print(f"   ✅ [2/6] Button")
                        await asyncio.sleep(1)
                        return True
                except:
                    pass

                # 3️⃣ Menuitem
                try:
                    item = page.get_by_role("menuitem", name="1080P").first
                    if await item.count() > 0:
                        await item.click(timeout=3000)
                        print(f"   ✅ [3/6] Menuitem")
                        await asyncio.sleep(1)
                        return True
                except:
                    pass

                # 4️⃣ Testo esatto
                try:
                    text_elem = page.get_by_text("1080P", exact=True).first
                    if await text_elem.count() > 0:
                        await text_elem.click(timeout=3000)
                        print(f"   ✅ [4/6] Testo")
                        await asyncio.sleep(1)
                        return True
                except:
                    pass

                # 5️⃣ FORZA BRUTA - Cerca TUTTI
                try:
                    all_visible = await page.locator('*:visible').all()
                    candidates = []

                    for elem in all_visible:
                        try:
                            text = await elem.text_content()
                            if not text or text.strip() != "1080P":
                                continue

                            box = await elem.bounding_box()
                            if not box or box['y'] < 600:
                                continue

                            candidates.append((elem, box))
                        except:
                            continue

                    if candidates:
                        candidates.sort(key=lambda c: c[1]['y'])
                        best_elem, best_box = candidates[0]

                        # Prova click
                        try:
                            await best_elem.click(force=True, timeout=3000)
                            print(f"   ✅ [5/6] Forza bruta")
                            await asyncio.sleep(1)
                            return True
                        except:
                            pass

                        # Prova mouse
                        try:
                            await page.mouse.click(
                                best_box['x'] + best_box['width'] / 2,
                                best_box['y'] + best_box['height'] / 2
                            )
                            print(f"   ✅ [5/6] Mouse click")
                            await asyncio.sleep(1)
                            return True
                        except:
                            pass
                except:
                    pass

                # 6️⃣ TASTIERA (480P → Down → 720P → Down → 1080P)
                try:
                    await page.keyboard.press('ArrowDown')
                    await asyncio.sleep(0.3)
                    await page.keyboard.press('ArrowDown')
                    await asyncio.sleep(0.3)
                    await page.keyboard.press('Enter')
                    print(f"   ✅ [6/6] Tastiera")
                    await asyncio.sleep(1)
                    return True
                except:
                    pass

                return False

            # ➡️ PROVA TUTTE LE STRATEGIE
            success = await try_click_1080()

            if not success:
                await page.keyboard.press('Escape')
                print(f"   ❌ Tutte le strategie fallite")
                return True

            # ➡️ VERIFICA
            await asyncio.sleep(2)
            all_buttons_check = await page.locator('button:visible').all()

            for btn in all_buttons_check:
                try:
                    text = await btn.text_content()
                    if not text:
                        continue
                    text_clean = text.strip()
                    box = await btn.bounding_box()
                    if not box or not (900 < box['y'] < 1100):
                        continue
                    if 'p' in text_clean.lower() and len(text_clean) < 8:
                        if '1080' in text_clean:
                            print(f"   ✅ {resolution} ATTIVATO!")
                            return True
                        else:
                            print(f"   ⚠️ Ancora {text_clean} (continuo)")
                            return True
                except:
                    continue

            return True

        except Exception as e:
            print(f"   ⚠️ Errore: {str(e)[:80]}")
            return True

    async def configure_generation_settings(page):
        if DEBUG_MODE:
            print(f"   ⚙️ Configurazione parametri...")
        await close_any_dialogs(page)
        await select_grok_model(page)
        await set_duration(page, DURATION)
        await set_aspect_ratio(page, ASPECT_RATIO)
        await set_resolution(page, RESOLUTION)
        await close_any_dialogs(page)

    async def insert_prompt_multiline(page, prompt_text):
        try:
            await close_any_dialogs(page)
            await asyncio.sleep(1)
            textarea = page.locator('textarea[maxlength="5000"]').first
            await textarea.wait_for(state='visible', timeout=10000)
            await textarea.click(force=True)
            await asyncio.sleep(0.5)
            await page.keyboard.press('Control+A')
            await page.keyboard.press('Backspace')
            await asyncio.sleep(0.3)
            await textarea.fill(prompt_text)
            await asyncio.sleep(0.5)
            return True
        except Exception as e:
            try:
                await page.keyboard.press('Escape')
                await asyncio.sleep(1)
                textarea = page.locator('textarea').first
                await textarea.click(force=True, timeout=5000)
                await asyncio.sleep(0.5)
                await page.keyboard.type(prompt_text, delay=10)
                await asyncio.sleep(0.5)
                return True
            except:
                return False

    async def click_generate_button(page):
        try:
            await close_any_dialogs(page)
            generate_selectors = [
                'button[class*="bg-light"][class*="font-medium"]:has(svg)',
                'button.bg-light:has(svg)',
                'button[data-slot="button"]:has(svg)'
            ]
            for selector in generate_selectors:
                try:
                    btn = page.locator(selector).last
                    if await btn.count() > 0 and await btn.is_visible():
                        await asyncio.sleep(2)
                        is_disabled = await btn.get_attribute('disabled')
                        if is_disabled is None:
                            await btn.click()
                            return True
                except:
                    continue
            await page.keyboard.press('Control+Enter')
            return True
        except Exception as e:
            return False

    async def is_button_enabled(btn):
        try:
            is_disabled = await btn.get_attribute('disabled')
            if is_disabled is not None:
                return False
            is_visible = await btn.is_visible()
            if not is_visible:
                return False
            box = await btn.bounding_box()
            if not box:
                return False
            return True
        except:
            return False

    async def wait_for_button_enabled(btn, timeout=30):
        start = asyncio.get_event_loop().time()
        while (asyncio.get_event_loop().time() - start) < timeout:
            if await is_button_enabled(btn):
                return True
            await asyncio.sleep(1)
        return False

    async def count_download_buttons(page, show_details=False):
        """🔢 Conta bottoni download"""
        try:
            await asyncio.sleep(2)
            download_buttons = []

            all_buttons = await page.locator('button:visible').all()

            for btn in all_buttons:
                try:
                    is_visible = await btn.is_visible()
                    if not is_visible:
                        continue

                    box = await btn.bounding_box()
                    if not box:
                        continue

                    if not (15 < box['width'] < 100 and 15 < box['height'] < 100):
                        continue

                    html = await btn.inner_html()
                    if 'M228,144v64a12,12,0,0,1-12,12H40' in html or 'download' in html.lower():
                        download_buttons.append((btn, box['x'], box['y']))
                        if show_details and DEBUG_MODE:
                            print(f"      ✓ Download @ ({int(box['x'])}, {int(box['y'])})")
                except:
                    continue

            download_buttons.sort(key=lambda b: (b[2], b[1]))

            return download_buttons
        except Exception as e:
            return []

    async def find_download_buttons_new_interface(page, check_enabled=True):
        try:
            buttons = await count_download_buttons(page, show_details=False)
            if check_enabled:
                enabled = []
                for btn, x, y in buttons:
                    if await is_button_enabled(btn):
                        enabled.append((btn, x, y))
                return enabled
            return buttons
        except:
            return []

    async def wait_for_new_videos(page, initial_count, expected_new, timeout=MAX_GENERATION_WAIT):
        """🎯 Aspetta NUOVI video"""
        print(f"\n⏳ Attendo {expected_new} nuovi video (da {initial_count} a {initial_count + expected_new})...")

        start = asyncio.get_event_loop().time()
        max_seen = initial_count
        last_print_time = 0

        while (asyncio.get_event_loop().time() - start) < timeout:
            await asyncio.sleep(10)

            current_buttons = await count_download_buttons(page, show_details=False)
            current_count = len(current_buttons)

            if current_count > max_seen:
                max_seen = current_count
                new_count = current_count - initial_count
                print(f"   📊 Rilevati: {current_count} (+{new_count} nuovi)")

            elapsed = int(asyncio.get_event_loop().time() - start)

            if current_count >= (initial_count + expected_new):
                new_videos = current_count - initial_count
                print(f"   ✅ {new_videos} nuovi video pronti!\n")
                return True, current_buttons

            if elapsed - last_print_time >= 30:
                print(f"   ⏳ {elapsed}s... {current_count}/{initial_count + expected_new}")
                last_print_time = elapsed

        print(f"   ⚠️ Timeout! Max visti: {max_seen}\n")
        final_buttons = await count_download_buttons(page, show_details=False)
        return False, final_buttons

    async def download_from_position(page, file_name, download_folder, position):
        """🎯 Download da posizione"""
        max_retries = MAX_DOWNLOAD_RETRIES
        for attempt in range(max_retries):
            try:
                if attempt > 0:
                    print(f"      🔄 Retry {attempt + 1}/{max_retries}...")
                    await asyncio.sleep(10)

                download_buttons = await find_download_buttons_new_interface(page, check_enabled=True)
                if not download_buttons:
                    if attempt < max_retries - 1:
                        continue
                    return False, "No buttons", None

                if position >= len(download_buttons):
                    if attempt < max_retries - 1:
                        continue
                    return False, f"Pos {position} invalid", None

                download_btn, x, y = download_buttons[position]

                if not await wait_for_button_enabled(download_btn, timeout=30):
                    if attempt < max_retries - 1:
                        continue
                    return False, "Not clickable", None

                async with page.expect_download(timeout=90000) as dl_info:
                    await download_btn.click()
                download = await dl_info.value
                ext = Path(download.suggested_filename).suffix or '.mp4'
                final_path = download_folder / f"{file_name}{ext}"
                await download.save_as(str(final_path))
                size = final_path.stat().st_size
                if size < 10000:
                    final_path.unlink()
                    if attempt < max_retries - 1:
                        continue
                    return False, "Corrupt", None
                size_mb = size / (1024 * 1024)
                return True, f"{size_mb:.1f} MB", final_path
            except Exception as e:
                if attempt < max_retries - 1:
                    continue
                return False, str(e)[:40], None
        return False, "Failed", None

    def check_video_exists(file_name, download_folder):
        for ext in ['.mp4', '.webm', '.mov']:
            path = download_folder / f"{file_name}{ext}"
            if path.exists():
                size_mb = path.stat().st_size / (1024 * 1024)
                return True, path, size_mb
        return False, None, 0

    def save_tracking(completed, failed):
        with open(tracking_file, 'w') as f:
            json.dump(list(completed), f)
        with open(failed_file, 'w') as f:
            json.dump(failed, f, indent=2)

    print("🌐 Avvio browser...")
    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=True,
            args=['--no-sandbox', '--disable-dev-shm-usage']
        )
        context = await browser.new_context(
            viewport={'width': 1920, 'height': 1080},
            accept_downloads=True,
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        )
        page = await context.new_page()
        page.on("dialog", lambda dialog: dialog.accept())

        try:
            if not cookies_file.exists():
                print("❌ Cookies non trovati")
                return

            print("📥 Carico sessione...")
            with open(cookies_file, 'r') as f:
                cookies = json.load(f)
            await context.add_cookies(cookies)

            print(f"🌐 Navigo a DiGen.ai...")
            await page.goto(BASE_URL, timeout=30000)
            await asyncio.sleep(3)

            await configure_generation_settings(page)
            print("✅ Sessione attiva!\n")

            print(f"🎬 INIZIO GENERAZIONE - Batch da {BATCH_SIZE}")
            print(f"⏱️ Pausa: {PAUSE_BETWEEN_PROMPTS}s | Retry: {MAX_DOWNLOAD_RETRIES} | Timeout: {MAX_GENERATION_WAIT//60}min")
            print("="*60 + "\n")

            batch_num = 0
            for i in range(0, total_prompts, BATCH_SIZE):
                batch_num += 1
                batch = df.iloc[i:i+BATCH_SIZE]

                to_generate = []
                for idx, row in batch.iterrows():
                    file_name = str(row['NomeFile']).strip()

                    if file_name in completed_prompts:
                        continue

                    exists, path, size = check_video_exists(file_name, download_folder)
                    if exists:
                        print(f"   ⏭️ Skip {file_name} ({size:.1f} MB)")
                        completed_prompts.add(file_name)
                        save_tracking(completed_prompts, failed_prompts)
                        continue

                    to_generate.append((idx, row))

                if not to_generate:
                    print(f"📦 BATCH {batch_num}: Completato\n")
                    continue

                print(f"📦 BATCH {batch_num}/{(total_prompts + BATCH_SIZE - 1) // BATCH_SIZE} - {len(to_generate)} video")

                await page.goto(BASE_URL, timeout=30000)
                await asyncio.sleep(2)
                await configure_generation_settings(page)
                await asyncio.sleep(2)

                initial_buttons = await count_download_buttons(page, show_details=False)
                initial_count = len(initial_buttons)
                print(f"   📊 Video presenti: {initial_count}")

                generation_success = {}
                for gen_idx, (idx, row) in enumerate(to_generate):
                    prompt_text = str(row['Prompt']).strip()
                    file_name = str(row['NomeFile']).strip()

                    print(f"   📝 {gen_idx + 1}/{len(to_generate)}: {file_name}...", end=" ")

                    if not await insert_prompt_multiline(page, prompt_text):
                        print("❌")
                        generation_success[file_name] = False
                        continue

                    if await click_generate_button(page):
                        print("✅")
                        generation_success[file_name] = True
                    else:
                        print("❌")
                        generation_success[file_name] = False

                    if gen_idx < len(to_generate) - 1:
                        await asyncio.sleep(PAUSE_BETWEEN_PROMPTS)

                expected_videos = sum(1 for success in generation_success.values() if success)

                if expected_videos == 0:
                    print(f"   ⚠️ Nessun video generato\n")
                    continue

                success_wait, all_buttons = await wait_for_new_videos(page, initial_count, expected_videos)

                await asyncio.sleep(5)

                final_buttons = await count_download_buttons(page, show_details=False)
                final_count = len(final_buttons)
                print(f"   📊 Video totali: {final_count} (+{final_count - initial_count})\n")

                successful_downloads = 0
                for download_idx in range(len(to_generate)):
                    idx, row = to_generate[download_idx]
                    file_name = str(row['NomeFile']).strip()

                    if not generation_success.get(file_name, False):
                        print(f"   {download_idx + 1}/{len(to_generate)}: {file_name} - ⏭️ Skip")
                        failed_prompts[file_name] = failed_prompts.get(file_name, 0) + 1
                        continue

                    # 🔥 FIX: Inverti l'ordine perché i video nuovi appaiono in alto
                    absolute_position = len(to_generate) - 1 - download_idx

                    print(f"   {download_idx + 1}/{len(to_generate)}: {file_name}...", end=" ")
                    success_dl, msg, path = await download_from_position(
                        page, file_name, download_folder, absolute_position
                    )

                    if success_dl:
                        print(f"✅ {msg}")
                        completed_prompts.add(file_name)
                        failed_prompts.pop(file_name, None)
                        successful_downloads += 1
                    else:
                        print(f"❌ {msg}")
                        failed_prompts[file_name] = failed_prompts.get(file_name, 0) + 1

                    save_tracking(completed_prompts, failed_prompts)
                    await asyncio.sleep(3)

                print(f"\n✅ BATCH {batch_num} OK ({successful_downloads}/{len(to_generate)})\n")

                if i + BATCH_SIZE < total_prompts:
                    await asyncio.sleep(15)

            print("\n" + "="*60)
            print("🎉 COMPLETATO!")
            print("="*60)

            videos = list(download_folder.glob("*.mp4")) + list(download_folder.glob("*.webm"))
            total_mb = sum(v.stat().st_size for v in videos) / (1024 * 1024)

            print(f"\n📊 STATISTICHE:")
            print(f"   ✅ Video: {len(videos)}/{total_prompts}")
            print(f"   💾 Spazio: {total_mb:.1f} MB")

            if failed_prompts:
                print(f"\n⚠️ FALLITI: {len(failed_prompts)}")
                for name, count in failed_prompts.items():
                    print(f"   - {name} ({count}x)")

            print(f"\n📁 {download_folder}\n")

        except Exception as e:
            print(f"\n❌ ERRORE: {e}")
            import traceback
            traceback.print_exc()
        finally:
            await browser.close()

await automate_digen_grok_batch()

In [ ]:
# ============================================
# CELLA 4: FIX FINALE - Monitora Cambiamenti Posizioni
# ============================================

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from playwright.async_api import async_playwright
import asyncio
import json
from pathlib import Path
from collections import defaultdict

MODEL = "veo"
BATCH_SIZE = 2
PAUSE_BETWEEN_PROMPTS = 20
MAX_DOWNLOAD_RETRIES = 2
MAX_GENERATION_WAIT = 500  # 7 minuti per generazioni lente
DOWNLOAD_FOLDER_NAME = "video_generati"

async def automate_digen_batch():
    if MODEL not in ["veo", "sora2"]:
        print("❌ MODEL deve essere 'veo' o 'sora2'")
        return

    BASE_URL = "https://digen.ai/"
    print(f"🎬 Modalità selezionata: {MODEL.upper()}\n")

    print("📂 Lettura file Excel...")
    df = pd.read_excel('prompts.xlsx')
    total_prompts = len(df)
    print(f"✅ Trovati {total_prompts} prompt\n")

    download_folder = Path(f"/content/drive/MyDrive/{DOWNLOAD_FOLDER_NAME}")
    download_folder.mkdir(exist_ok=True, parents=True)
    print(f"📁 Cartella download: {download_folder}\n")

    cookies_file = Path("digen_cookies.json")
    tracking_file = Path(f"video_tracking_{MODEL}.json")
    failed_file = Path(f"failed_prompts_{MODEL}.json")

    if tracking_file.exists():
        with open(tracking_file, 'r') as f:
            completed_prompts = set(json.load(f))
    else:
        completed_prompts = set()

    if failed_file.exists():
        with open(failed_file, 'r') as f:
            failed_prompts = json.load(f)
    else:
        failed_prompts = {}

    # ============================================
    # FUNZIONI HELPER
    # ============================================

    async def select_model(page, model):
        try:
            print(f"   🎯 Seleziono modello {model.upper()}...")

            if model == "sora2":
                selectors = [
                    'a[href*="sora"]',
                    'button:has-text("Sora")',
                    'a:has-text("Sora2")',
                    '[data-model="sora"]',
                    'nav a[href="/sora"]'
                ]

                for selector in selectors:
                    try:
                        element = page.locator(selector).first
                        if await element.count() > 0:
                            await element.click()
                            await asyncio.sleep(2)
                            print(f"   ✅ Tab SORA2 selezionato")

                            current_url = page.url
                            if 'sora' in current_url.lower():
                                if 'seconds=' not in current_url:
                                    await page.goto(f"{current_url}?seconds=15&orientation=landscape", timeout=30000)
                                    await asyncio.sleep(2)
                                return True
                    except:
                        continue

                print(f"   ⚠️ Tab non trovato, navigo direttamente...")
                await page.goto("https://digen.ai/sora?seconds=15&orientation=landscape", timeout=30000)
                await asyncio.sleep(3)
                return True

            elif model == "veo":
                selectors = [
                    'a[href*="veo"]',
                    'button:has-text("VEO")',
                    'a:has-text("VEO")',
                    '[data-model="veo"]',
                    'nav a[href="/veo"]'
                ]

                for selector in selectors:
                    try:
                        element = page.locator(selector).first
                        if await element.count() > 0:
                            await element.click()
                            await asyncio.sleep(2)
                            print(f"   ✅ Tab VEO selezionato")

                            current_url = page.url
                            if 'veo' in current_url.lower():
                                if 'orientation=' not in current_url:
                                    await page.goto(f"{current_url}?orientation=landscape", timeout=30000)
                                    await asyncio.sleep(2)
                                return True
                    except:
                        continue

                print(f"   ⚠️ Tab non trovato, navigo direttamente...")
                await page.goto("https://digen.ai/veo?orientation=landscape", timeout=30000)
                await asyncio.sleep(3)
                return True

            return False

        except Exception as e:
            print(f"   ⚠️ Errore selezione modello: {e}")
            return False

    async def verify_model_selected(page, model):
        try:
            current_url = page.url.lower()

            if model == "sora2" and "sora" in current_url:
                print(f"   ✅ Modello SORA2 confermato (URL: {page.url})")
                return True
            elif model == "veo" and "veo" in current_url:
                print(f"   ✅ Modello VEO confermato (URL: {page.url})")
                return True
            else:
                print(f"   ⚠️ Modello non confermato! URL: {page.url}")
                return False
        except:
            return False

    async def insert_prompt_multiline(page, prompt_text):
        try:
            textarea = page.locator('textarea').first
            await textarea.click()
            await asyncio.sleep(0.3)
            await page.keyboard.press('Control+A')
            await page.keyboard.press('Backspace')
            await asyncio.sleep(0.3)
            await textarea.fill(prompt_text)
            await asyncio.sleep(0.5)
            return True
        except Exception as e:
            print(f"      ⚠️ Errore inserimento: {e}")
            return False

    async def is_button_enabled(btn):
        try:
            is_disabled = await btn.get_attribute('disabled')
            if is_disabled is not None:
                return False

            is_visible = await btn.is_visible()
            if not is_visible:
                return False

            box = await btn.bounding_box()
            if not box:
                return False

            return True
        except:
            return False

    async def wait_for_button_enabled(btn, timeout=30):
        start = asyncio.get_event_loop().time()

        while (asyncio.get_event_loop().time() - start) < timeout:
            if await is_button_enabled(btn):
                return True
            await asyncio.sleep(1)

        return False

    async def find_download_buttons_in_sidebar(page, check_enabled=True):
        try:
            await asyncio.sleep(2)
            all_buttons = await page.locator('button:visible').all()
            sidebar_buttons = []

            for btn in all_buttons:
                try:
                    is_visible = await btn.is_visible()
                    if not is_visible:
                        continue

                    box = await btn.bounding_box()
                    if not box:
                        continue

                    if box['x'] < 1500:
                        continue

                    if check_enabled:
                        if not await is_button_enabled(btn):
                            continue

                    html = await btn.inner_html()
                    has_svg = 'svg' in html.lower()
                    is_icon_button = (box['width'] < 100 and box['height'] < 100)

                    if has_svg and is_icon_button:
                        sidebar_buttons.append((btn, box['x'], box['y']))

                except Exception as e:
                    continue

            by_y = defaultdict(list)
            for btn, x, y in sidebar_buttons:
                y_group = round(y / 30) * 30
                by_y[y_group].append((btn, x, y))

            download_buttons = []
            for y_group in sorted(by_y.keys()):
                buttons = by_y[y_group]
                buttons.sort(key=lambda b: b[1])
                rightmost = buttons[-1]
                download_buttons.append(rightmost)

            return download_buttons

        except Exception as e:
            print(f"      ⚠️ Errore ricerca bottoni: {e}")
            return []

    # 🆕 Monitora CAMBIAMENTI nelle posizioni Y (nuovi video sostituiscono vecchi)
    async def wait_for_videos_ready(page, initial_clickable, expected_new, timeout=MAX_GENERATION_WAIT):
        print(f"   📊 Bottoni clickable iniziali: {initial_clickable}")
        print(f"   🎯 Nuovi video attesi: {expected_new}")
        print(f"   ⏰ Timeout massimo: {timeout}s (~{timeout//60} min)\n")

        # 🆕 Ottieni snapshot posizioni iniziali
        initial_buttons = await find_download_buttons_in_sidebar(page, check_enabled=True)
        initial_positions = set((int(y//50)*50) for _, x, y in initial_buttons)  # Y arrotondate a 50px

        print(f"   📍 Posizioni Y iniziali: {sorted(initial_positions)}")

        start = asyncio.get_event_loop().time()
        changes_detected = 0
        last_positions = initial_positions

        while (asyncio.get_event_loop().time() - start) < timeout:
            await asyncio.sleep(10)

            current_buttons = await find_download_buttons_in_sidebar(page, check_enabled=True)
            current_positions = set((int(y//50)*50) for _, x, y in current_buttons)
            current_clickable = len(current_buttons)

            elapsed = int(asyncio.get_event_loop().time() - start)

            # 🆕 Rileva CAMBIAMENTI nelle posizioni
            if current_positions != last_positions:
                changes_detected += 1
                print(f"      🔄 {elapsed}s: Posizioni cambiate! (cambio #{changes_detected})")
                print(f"      📍 Nuove posizioni Y: {sorted(current_positions)}")

                # Se ha rilevato abbastanza cambiamenti = video pronti
                if changes_detected >= expected_new:
                    print(f"\n   ✅ Rilevati {changes_detected} cambiamenti = {expected_new} video pronti!")
                    return True

                # Aggiorna snapshot
                last_positions = current_positions

            # Log ogni 30s
            if elapsed % 30 == 0 and elapsed > 0:
                print(f"      ⏳ {elapsed}s... clickable: {current_clickable}, cambiamenti: {changes_detected}/{expected_new}")

            # ✅ Dopo 90s con cambiamenti rilevati = probabilmente pronti
            if elapsed > 90 and changes_detected >= expected_new:
                print(f"\n   ✅ {changes_detected} cambiamenti dopo {elapsed}s, procedo!")
                return True

        # Timeout
        print(f"\n   ⚠️ Timeout {timeout}s! Cambiamenti: {changes_detected}/{expected_new}")

        if changes_detected > 0:
            print(f"   ✅ Rilevati {changes_detected} cambiamenti, video probabilmente pronti!")
            return True
        else:
            print(f"   ⚠️ Nessun cambiamento rilevato, procedo con retry...")
            return False

    async def download_from_sidebar_position(page, file_name, download_folder, position):
        max_retries = MAX_DOWNLOAD_RETRIES

        for attempt in range(max_retries):
            try:
                if attempt > 0:
                    print(f"      🔄 Tentativo {attempt + 1}/{max_retries}...")
                    await asyncio.sleep(10)

                print(f"      🔍 Cerco bottoni download...")
                download_buttons = await find_download_buttons_in_sidebar(page, check_enabled=True)

                if not download_buttons:
                    if attempt < max_retries - 1:
                        print(f"      ⚠️ Nessun bottone clickable, attendo...")
                        continue
                    return False, "Nessun bottone trovato dopo retry", None

                print(f"      ✅ Trovati {len(download_buttons)} bottoni clickable")

                if position >= len(download_buttons):
                    if attempt < max_retries - 1:
                        print(f"      ⚠️ Posizione {position} non esiste, attendo...")
                        continue
                    return False, f"Posizione {position} non esiste (solo {len(download_buttons)} bottoni)", None

                download_btn, x, y = download_buttons[position]
                print(f"      🎯 Bottone posizione {position} a ({int(x)}, {int(y)})")

                print(f"      ⏳ Verifico che il bottone sia clickable...")
                if not await wait_for_button_enabled(download_btn, timeout=30):
                    if attempt < max_retries - 1:
                        print(f"      ⚠️ Bottone non clickable, riprovo...")
                        continue
                    return False, "Bottone non clickable dopo 30s", None

                print(f"      📥 Clicco download...")
                async with page.expect_download(timeout=90000) as dl_info:
                    await download_btn.click()
                download = await dl_info.value

                ext = Path(download.suggested_filename).suffix or '.mp4'
                final_path = download_folder / f"{file_name}{ext}"

                print(f"      💾 Salvo in Drive...")
                await download.save_as(str(final_path))

                size = final_path.stat().st_size
                if size < 10000:
                    final_path.unlink()
                    if attempt < max_retries - 1:
                        print(f"      ⚠️ File corrotto, riprovo...")
                        continue
                    return False, "File corrotto", None

                size_mb = size / (1024 * 1024)
                return True, f"{size_mb:.1f} MB", final_path

            except Exception as e:
                if attempt < max_retries - 1:
                    print(f"      ⚠️ Errore: {str(e)[:50]}, riprovo...")
                    continue
                return False, f"Errore: {str(e)[:100]}", None

        return False, "Tutti i tentativi falliti", None

    def check_video_exists(file_name, download_folder):
        for ext in ['.mp4', '.webm', '.mov']:
            path = download_folder / f"{file_name}{ext}"
            if path.exists():
                size_mb = path.stat().st_size / (1024 * 1024)
                return True, path, size_mb
        return False, None, 0

    def save_tracking(completed, failed):
        with open(tracking_file, 'w') as f:
            json.dump(list(completed), f)
        with open(failed_file, 'w') as f:
            json.dump(failed, f, indent=2)

    # ============================================
    # MAIN EXECUTION
    # ============================================

    print("🌐 Avvio browser...")
    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=True,
            args=['--no-sandbox', '--disable-dev-shm-usage']
        )
        context = await browser.new_context(
            viewport={'width': 1920, 'height': 1080},
            accept_downloads=True,
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        )
        page = await context.new_page()
        page.on("dialog", lambda dialog: dialog.accept())

        try:
            if not cookies_file.exists():
                print("❌ File cookies non trovato! Esegui prima la CELLA 3")
                return

            print("📥 Carico sessione...")
            with open(cookies_file, 'r') as f:
                cookies = json.load(f)
            await context.add_cookies(cookies)

            print(f"🌐 Navigo a DiGen.ai...")
            await page.goto(BASE_URL, timeout=30000)
            await asyncio.sleep(3)

            if not await select_model(page, MODEL):
                print("❌ Errore nella selezione del modello!")
                return

            if not await verify_model_selected(page, MODEL):
                print("❌ Modello non confermato, esco...")
                return

            print("✅ Sessione valida e modello selezionato!\n")

            print(f"🎬 INIZIO GENERAZIONE ({MODEL.upper()}) - Batch da {BATCH_SIZE}")
            print(f"⏱️ Pausa tra prompt: {PAUSE_BETWEEN_PROMPTS}s")
            print(f"🔄 Retry download: {MAX_DOWNLOAD_RETRIES} tentativi")
            print(f"⏰ Timeout generazione: {MAX_GENERATION_WAIT}s (~{MAX_GENERATION_WAIT//60} min)")
            print("="*60 + "\n")

            batch_num = 0
            for i in range(0, total_prompts, BATCH_SIZE):
                batch_num += 1
                batch = df.iloc[i:i+BATCH_SIZE]

                to_generate = []
                for idx, row in batch.iterrows():
                    file_name = str(row['NomeFile']).strip()

                    if file_name in completed_prompts:
                        continue

                    exists, path, size = check_video_exists(file_name, download_folder)
                    if exists:
                        print(f"   ⏭️ Skip {file_name}: già presente ({size:.1f} MB)")
                        completed_prompts.add(file_name)
                        save_tracking(completed_prompts, failed_prompts)
                        continue

                    to_generate.append((idx, row))

                if not to_generate:
                    print(f"📦 BATCH {batch_num}: Tutti già completati, skip\n")
                    continue

                print(f"📦 BATCH {batch_num}/{(total_prompts + BATCH_SIZE - 1) // BATCH_SIZE}")
                print(f"   Video da generare: {len(to_generate)}")

                print(f"   🔄 Ricarico pagina...")
                await page.goto(BASE_URL, timeout=30000)
                await asyncio.sleep(2)
                await select_model(page, MODEL)
                await asyncio.sleep(2)

                # Conta bottoni clickable iniziali
                initial_buttons = await find_download_buttons_in_sidebar(page, check_enabled=True)
                initial_clickable = len(initial_buttons)

                generation_success = {}
                for gen_idx, (idx, row) in enumerate(to_generate):
                    prompt_text = str(row['Prompt']).strip()
                    file_name = str(row['NomeFile']).strip()

                    print(f"\n   📝 {gen_idx + 1}/{len(to_generate)}: {file_name}")

                    if not await insert_prompt_multiline(page, prompt_text):
                        print(f"   ⚠️ Errore inserimento, salto")
                        generation_success[file_name] = False
                        continue

                    print(f"   ✅ Inserito")
                    print(f"   🚀 Generate...")
                    await page.keyboard.press('Control+Enter')
                    print(f"   ✅ Inviato")
                    generation_success[file_name] = True

                    if gen_idx < len(to_generate) - 1:
                        print(f"   ⏸️ Pausa {PAUSE_BETWEEN_PROMPTS}s...")
                        await asyncio.sleep(PAUSE_BETWEEN_PROMPTS)

                expected_videos = sum(1 for success in generation_success.values() if success)

                if expected_videos == 0:
                    print(f"\n⚠️ Nessun video da generare in questo batch\n")
                    continue

                # 🆕 Attendi rilevando CAMBIAMENTI nelle posizioni
                print(f"\n⏳ Attendo che {expected_videos} video diventino pronti...")
                await wait_for_videos_ready(page, initial_clickable, expected_videos)

                await asyncio.sleep(5)

                print(f"\n📥 Scarico {len(to_generate)} video...\n")

                successful_downloads = 0
                for download_idx in range(len(to_generate)):
                    idx, row = to_generate[download_idx]
                    file_name = str(row['NomeFile']).strip()

                    if not generation_success.get(file_name, False):
                        print(f"   ⏭️ {download_idx + 1}/{len(to_generate)}: {file_name} - Non generato, skip")
                        failed_prompts[file_name] = failed_prompts.get(file_name, 0) + 1
                        continue

                    position = len(to_generate) - 1 - download_idx

                    print(f"   {download_idx + 1}/{len(to_generate)}: {file_name} (pos: {position})")
                    success, msg, path = await download_from_sidebar_position(
                        page, file_name, download_folder, position
                    )

                    if success:
                        print(f"      ✅ {msg}\n")
                        completed_prompts.add(file_name)
                        failed_prompts.pop(file_name, None)
                        successful_downloads += 1
                    else:
                        print(f"      ❌ {msg}\n")
                        failed_prompts[file_name] = failed_prompts.get(file_name, 0) + 1

                    save_tracking(completed_prompts, failed_prompts)
                    await asyncio.sleep(3)

                print(f"✅ BATCH {batch_num} COMPLETATO! ({successful_downloads}/{len(to_generate)} scaricati)\n")

                if i + BATCH_SIZE < total_prompts:
                    print("⏸️ Pausa 15s prima del prossimo batch...")
                    await asyncio.sleep(15)

            print("\n" + "="*60)
            print("🎉 GENERAZIONE COMPLETATA!")
            print("="*60)

            videos = list(download_folder.glob("*.mp4")) + list(download_folder.glob("*.webm"))
            total_mb = sum(v.stat().st_size for v in videos) / (1024 * 1024)

            print(f"\n📊 STATISTICHE FINALI:")
            print(f"   ✅ Video scaricati: {len(videos)}/{total_prompts}")
            print(f"   💾 Spazio totale: {total_mb:.1f} MB")
            print(f"   ⚡ Batch processati: {batch_num}")
            print(f"   🎬 Modello usato: {MODEL.upper()}")

            if failed_prompts:
                print(f"\n⚠️ VIDEO FALLITI: {len(failed_prompts)}")
                print("📝 Lista (salvata in failed_prompts.json):")
                for name, count in list(failed_prompts.items())[:10]:
                    print(f"   - {name}: {count} tentativi")
                if len(failed_prompts) > 10:
                    print(f"   ... e altri {len(failed_prompts) - 10}")
                print(f"\n💡 TIP: Riesegui lo script per riprovare i falliti")

            print(f"\n📁 Cartella: {download_folder}")
            print(f"📄 Tracking: {tracking_file}")
            print(f"❌ Fallimenti: {failed_file}\n")

        except Exception as e:
            print(f"\n❌ ERRORE CRITICO: {e}")
            import traceback
            traceback.print_exc()
        finally:
            await browser.close()

# ESEGUI
await automate_digen_batch()


Mounted at /content/drive
🎬 Modalità selezionata: VEO

📂 Lettura file Excel...
✅ Trovati 19 prompt

📁 Cartella download: /content/drive/MyDrive/video_generati

🌐 Avvio browser...
📥 Carico sessione...
🌐 Navigo a DiGen.ai...
   🎯 Seleziono modello VEO...
   ⚠️ Tab non trovato, navigo direttamente...
   ✅ Modello VEO confermato (URL: https://digen.ai/veo?orientation=landscape)
✅ Sessione valida e modello selezionato!

🎬 INIZIO GENERAZIONE (VEO) - Batch da 2
⏱️ Pausa tra prompt: 20s
🔄 Retry download: 2 tentativi
⏰ Timeout generazione: 500s (~8 min)

📦 BATCH 1/10
   Video da generare: 2
   🔄 Ricarico pagina...
   🎯 Seleziono modello VEO...
   ⚠️ Tab non trovato, navigo direttamente...

   📝 1/2: SCENE 03_01
   ✅ Inserito
   🚀 Generate...
   ✅ Inviato
   ⏸️ Pausa 20s...

   📝 2/2: SCENE 03_02
   ✅ Inserito
   🚀 Generate...
   ✅ Inviato

⏳ Attendo che 2 video diventino pronti...
   📊 Bottoni clickable iniziali: 5
   🎯 Nuovi video attesi: 2
   ⏰ Timeout massimo: 500s (~8 min)

   📍 Posizioni Y

Exception: Browser.close: Connection closed while reading from the driver

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# CELLA: Elimina screenshot vecchi
import shutil
from pathlib import Path

debug_folder = Path("debug_screenshots")

if debug_folder.exists():
    # Elimina cartella e tutto il contenuto
    shutil.rmtree(debug_folder)
    print(f"✅ Eliminati tutti gli screenshot da: {debug_folder}")

    # Ricrea cartella vuota
    debug_folder.mkdir()
    print(f"✅ Cartella debug ricreata vuota")
else:
    print(f"ℹ️ Cartella {debug_folder} non esiste")

# Verifica
screenshots = list(debug_folder.glob("*.png"))
print(f"\n📸 Screenshot rimasti: {len(screenshots)}")


✅ Eliminati tutti gli screenshot da: debug_screenshots
✅ Cartella debug ricreata vuota

📸 Screenshot rimasti: 0


In [ ]:
# CELLA 3.5: PULISCI TRACKING (usa solo se vuoi ricominciare da zero)
import json
from pathlib import Path

tracking_file = Path("video_tracking.json")

if tracking_file.exists():
    with open(tracking_file, 'r') as f:
        completed = json.load(f)

    print(f"📋 Video nel tracking: {len(completed)}")
    for v in completed:
        print(f"   • {v}")

    print("\n⚠️ Vuoi pulire il tracking? (ricomincerà da zero)")
    # Decommentare la riga sotto per pulire:
    # tracking_file.unlink()
    # print("✅ Tracking pulito!")
else:
    print("📋 Nessun tracking esistente")


📋 Nessun tracking esistente


In [ ]:
# CELLA SAVE: SALVA TUTTO SU GOOGLE DRIVE (esegui UNA VOLTA)

from google.colab import drive
drive.mount('/content/drive')

import shutil
from pathlib import Path

# Crea cartella progetto su Drive
project_folder = Path("/content/drive/MyDrive/VeoAutomation")
project_folder.mkdir(exist_ok=True)

print("📁 Creo struttura progetto su Google Drive...")

# Salva script Python
script_content = """
# SCRIPT VEO AUTOMATION
# Copia tutto il codice della CELLA 4 qui dentro
"""

with open(project_folder / "veo_automation.py", 'w') as f:
    f.write(script_content)

# Copia file importanti
files_to_save = [
    "prompts.xlsx",
    "digen_cookies.json",
    "video_tracking.json"
]

for file in files_to_save:
    if Path(file).exists():
        shutil.copy(file, project_folder / file)
        print(f"✅ Salvato: {file}")

print(f"\n🎉 Progetto salvato in: {project_folder}")
print("\n💡 Ogni giorno:")
print("1. Apri questo notebook")
print("2. Esegui: Runtime > Run all")
print("3. Aspetta che finisca")


Mounted at /content/drive
📁 Creo struttura progetto su Google Drive...
✅ Salvato: prompts.xlsx
✅ Salvato: digen_cookies.json

🎉 Progetto salvato in: /content/drive/MyDrive/VeoAutomation

💡 Ogni giorno:
1. Apri questo notebook
2. Esegui: Runtime > Run all
3. Aspetta che finisca
